<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/note_book/submission_71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ライブラリ

In [ ]:
!pip install optuna > /dev/null
!pip install pyproj > /dev/null
!pip install simdkalman > /dev/null
!pip install osmnx momepy geopandas > /dev/null
    
import os
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import interp1d

from shapely.geometry import Point
import osmnx as ox
import momepy
import geopandas as gpd
import optuna
import plotly
import plotly.express as px
import pyproj
from pathlib import Path
from pyproj import Proj, transform
from tqdm.notebook import tqdm
import simdkalman

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


#Data

In [ ]:
dir = Path("/content/drive/MyDrive/GSDC")
train_base = pd.read_csv(dir / "baseline_locations_train.csv")

#test_base = pd.read_csv(dir / "baseline_locations_test.csv")
test_base = pd.read_csv("/content/drive/MyDrive/GSDC/test_predict_next_point_SJC_2.csv")

sub = pd.read_csv(dir / "sample_submission.csv")


def get_groundtruth(path: Path) -> pd.DataFrame:
        output_df = pd.DataFrame()
        
        for path in glob(str(dir / 'train/*/*/ground_truth.csv')):
            _df = pd.read_csv(path)
            output_df = pd.concat([output_df, _df])
        output_df = output_df.reset_index(drop=True)
        
        _columns = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
        output_df[['t_'+col for col in _columns]] = output_df[_columns]
        output_df = output_df.drop(columns=_columns, axis=1)
        return output_df

train_base = train_base.merge(
    get_groundtruth(dir),
    on=['collectionName', 'phoneName', 'millisSinceGpsEpoch']
)

In [ ]:
train_area_5s = []
for area_5 in train_area_5:
    area_5_df = train_base[train_base["collectionName"] == area_5]
    train_area_5s.append(area_5_df)
area_5_df = pd.concat(train_area_5s)

test_area_5s = []
for area_5 in train_area_5:
    area_5_df = test_base[test_base["collectionName"] == area_5]
    test_area_5s.append(area_5_df)
area_5_df = pd.concat(test_area_5s)

In [ ]:
train_SJC_list = ["2021-04-22-US-SJC-1", "2021-04-28-US-SJC-1", "2021-04-29-US-SJC-2"]
test_SJC_list = ["2021-04-22-US-SJC-2", "2021-04-29-US-SJC-3"]

train_SJCs = []
for SJC in train_SJC_list:
    SJC_df = train_base[train_base["collectionName"] == SJC]
    train_SJCs.append(SJC_df)
train_SJC = pd.concat(train_SJCs)

test_SJCs = []
for SJC in test_SJC_list:
    SJC_df = test_base[test_base["collectionName"] == SJC]
    test_SJCs.append(SJC_df)
test_SJC = pd.concat(test_SJCs)

#共通関数

In [ ]:
def calc_haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(d**0.5)
    return dist

#defs

##mean points

In [ ]:
def add_mean_points_features(input_df):
    output_df = input_df.copy()
    length = len(output_df["phone"].values)
    output_df["latDeg_pro_1"] = output_df["latDeg"].shift(-1)
    output_df["latDeg_pro_2"] = output_df["latDeg"].shift(-2)
    output_df["lngDeg_pro_1"] = output_df["lngDeg"].shift(-1)
    output_df["lngDeg_pro_2"] = output_df["lngDeg"].shift(-2)
    output_df["millisSinceGpsEpoch_pro_1"] = output_df["millisSinceGpsEpoch"].shift(-1)
    output_df["millisSinceGpsEpoch_pro_2"] = output_df["millisSinceGpsEpoch"].shift(-2)
    output_df["latDeg_mean_point"] = (output_df["latDeg"] + ((output_df["latDeg_pro_2"] - output_df["latDeg"]) * 
                                                            ((output_df["millisSinceGpsEpoch_pro_1"] - output_df["millisSinceGpsEpoch"]) /
                                                            (output_df["millisSinceGpsEpoch_pro_2"] - output_df["millisSinceGpsEpoch"])))).shift(1)
        
    output_df["lngDeg_mean_point"] = (output_df["lngDeg"] + ((output_df["lngDeg_pro_2"] - output_df["lngDeg"]) * 
                                                            ((output_df["millisSinceGpsEpoch_pro_1"] - output_df["millisSinceGpsEpoch"]) /
                                                            (output_df["millisSinceGpsEpoch_pro_2"] - output_df["millisSinceGpsEpoch"])))).shift(1)

    return output_df

In [ ]:
def mean_points(input_df):
    output_df = input_df.copy()

    lat_list = []
    lng_list = []

    for collection in output_df["collectionName"].unique():
        collection_df = output_df[output_df["collectionName"] == collection]
        if collection in output_df["collectionName"].unique():
            for phone in collection_df["phoneName"].unique():
                phone_df = collection_df[collection_df["phoneName"] == phone]
                mean_points_df = add_mean_points_features(phone_df)
                lat_head = mean_points_df["latDeg"].values[0]
                lng_head = mean_points_df["lngDeg"].values[0]
                lat_tail = mean_points_df["latDeg"].values[-1]
                lng_tail = mean_points_df["lngDeg"].values[-1]
                lat_tail_2 = mean_points_df["latDeg"].values[-2]
                lng_tail_2 = mean_points_df["lngDeg"].values[-2]
                for lat, lng, lat_1, lng_1, lat_2, lng_2, lat_mp, lng_mp in zip(
                    mean_points_df["latDeg"].to_numpy(),
                    mean_points_df["lngDeg"].to_numpy(),
                    mean_points_df["latDeg_pro_1"].to_numpy(),
                    mean_points_df["lngDeg_pro_1"].to_numpy(),
                    mean_points_df["latDeg_pro_2"].to_numpy(),
                    mean_points_df["lngDeg_pro_2"].to_numpy(),
                    mean_points_df["latDeg_mean_point"].to_numpy(),
                    mean_points_df["lngDeg_mean_point"].to_numpy()
                ):
                
                        p0 = np.array([lat, lng])
                        p1 = np.array([lat_1, lng_1])
                        p2 = np.array([lat_2, lng_2])
                        mp = np.array([lat_mp, lng_mp])

                        vec_p0_p2 = (p2 - p0) / np.linalg.norm(p2 - p0)
                        p0_mean_point = (p1 - p0) @ vec_p0_p2 * vec_p0_p2 + p0

                        diff_p1_mp = mp - p0_mean_point

                        lat_fixed = lat_1 + diff_p1_mp[0]
                        lng_fixed = lng_1 + diff_p1_mp[1]

                        if np.isnan(lat_fixed) == True:
                            lat_list.append(lat)
                            lng_list.append(lng)
                            continue

                        else:
                            lat_list.append(lat_fixed)
                            lng_list.append(lng_fixed)

    output_df["latDeg"] = lat_list
    output_df["lngDeg"] = lng_list
    #output_df["latDeg"] = output_df["latDeg"].shift(1)
    #output_df["lngDeg"] = output_df["lngDeg"].shift(1)
    return output_df[input_df.columns]

## make triangle

In [ ]:
#必要な特徴量を追加
def add_triangle_features(input_df):
    output_df = input_df.copy()
    output_df["latDeg_pro_1"] = output_df["latDeg"].shift(-1)
    output_df["latDeg_pro_2"] = output_df["latDeg"].shift(-2)
    output_df["lngDeg_pro_1"] = output_df["lngDeg"].shift(-1)
    output_df["lngDeg_pro_2"] = output_df["lngDeg"].shift(-2)
    output_df["millisSinceGpsEpoch_pro_1"] = output_df["millisSinceGpsEpoch"].shift(-1)
    output_df["millisSinceGpsEpoch_pro_2"] = output_df["millisSinceGpsEpoch"].shift(-2)
    output_df["latDeg_mean_point"] = (output_df["latDeg"] + ((output_df["latDeg_pro_2"] - output_df["latDeg"]) * 
                                                            ((output_df["millisSinceGpsEpoch_pro_1"] - output_df["millisSinceGpsEpoch"]) /
                                                            (output_df["millisSinceGpsEpoch_pro_2"] - output_df["millisSinceGpsEpoch"])))).shift(1)
        
    output_df["lngDeg_mean_point"] = (output_df["lngDeg"] + ((output_df["lngDeg_pro_2"] - output_df["lngDeg"]) * 
                                                            ((output_df["millisSinceGpsEpoch_pro_1"] - output_df["millisSinceGpsEpoch"]) /
                                                            (output_df["millisSinceGpsEpoch_pro_2"] - output_df["millisSinceGpsEpoch"])))).shift(1)


    degree_list = []
    for lat, lng, lat_1, lng_1, lat_2, lng_2 in zip(
        output_df["latDeg"].to_numpy(),
        output_df["lngDeg"].to_numpy(),
        output_df["latDeg_pro_1"].to_numpy(),
        output_df["lngDeg_pro_1"].to_numpy(),
        output_df["latDeg_pro_2"].to_numpy(),
        output_df["lngDeg_pro_2"].to_numpy()
    ):
        p0 = np.array([lat, lng])
        p1 = np.array([lat_1, lng_1])
        p2 = np.array([lat_2, lng_2])
            
        vec_p0 = p0 - p1
        vec_p2 = p2 - p1
        length_vec_p0 = np.linalg.norm(vec_p0)
        length_vec_p2 = np.linalg.norm(vec_p2)
        inner = np.inner(vec_p0, vec_p2)
        degree = np.rad2deg(np.arccos(inner / (length_vec_p0 * length_vec_p2)))
        degree_list.append(degree)
    degree_list.insert(0, 180)
    degree_list.pop(-1)
    degree_list[-1] = 180
    output_df["degree"] = degree_list
    return output_df

In [ ]:
#主な処理
def make_triangle(input_df):
    output_df = input_df.copy()

    lat_list = []
    lng_list = []

    for collection in output_df["collectionName"].unique():
        collection_df = output_df[output_df["collectionName"] == collection]
        if collection in output_df["collectionName"].unique():
            for phone in collection_df["phoneName"].unique():
                phone_df = collection_df[collection_df["phoneName"] == phone]
                triangle_df = add_triangle_features(phone_df)
                for lat, lng, lat_mp, lng_mp, deg in zip(
                    triangle_df["latDeg"].to_numpy(),
                    triangle_df["lngDeg"].to_numpy(),
                    triangle_df["latDeg_mean_point"].to_numpy(),
                    triangle_df["lngDeg_mean_point"].to_numpy(),
                    triangle_df["degree"].to_numpy()
                ):
                    if deg < 155:
                        lat_f = (lat + lat_mp)/2
                        lng_f = (lng + lng_mp)/2
                        lat_list.append(lat_f)
                        lng_list.append(lng_f)
                    else:
                        lat_list.append(lat)
                        lng_list.append(lng)
        else:
            for lat, lng in zip(
                collection_df["latDeg"].to_numpy(),
                collection_df["lngDeg"].to_numpy(),
            ):
                lat_list.append(lat)
                lng_list.append(lng)
    output_df["latDeg"] = lat_list
    output_df["lngDeg"] = lng_list
    return output_df[input_df.columns]

##outlier

In [ ]:
def outlier_train(input_df):
    output_df = input_df.copy()
    output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]

    output_df["dist_pre"] = 0
    output_df["dist_pro"] = 0

    output_df['latDeg_pre'] = output_df['latDeg'].shift(periods=1,fill_value=0)
    output_df['lngDeg_pre'] = output_df['lngDeg'].shift(periods=1,fill_value=0)
    output_df['latDeg_pro'] = output_df['latDeg'].shift(periods=-1,fill_value=0)
    output_df['lngDeg_pro'] = output_df['lngDeg'].shift(periods=-1,fill_value=0)
    output_df['dist_pre'] = calc_haversine(output_df.latDeg_pre, output_df.lngDeg_pre, output_df.latDeg, output_df.lngDeg)
    output_df['dist_pro'] = calc_haversine(output_df.latDeg, output_df.lngDeg, output_df.latDeg_pro, output_df.lngDeg_pro)

    list_phone = output_df['phone'].unique()
    for phone in list_phone:
        ind_s = output_df[train_base['phone'] == phone].index[0]
        ind_e = output_df[train_base['phone'] == phone].index[-1]
        output_df.loc[ind_s,'dist_pre'] = 0
        output_df.loc[ind_e,'dist_pro'] = 0

    pro_95 = output_df['dist_pro'].mean() + (output_df['dist_pro'].std() * 2)
    pre_95 = output_df['dist_pre'].mean() + (output_df['dist_pre'].std() * 2)
    ind = output_df[(output_df['dist_pro'] > pro_95)&(output_df['dist_pre'] > pre_95)][['dist_pre','dist_pro']].index

    for i in ind:
        output_df.loc[i,'latDeg'] = (output_df.loc[i-1,'latDeg'] + output_df.loc[i+1,'latDeg'])/2
        output_df.loc[i,'lngDeg'] = (output_df.loc[i-1,'lngDeg'] + output_df.loc[i+1,'lngDeg'])/2

    return output_df

In [ ]:
def outlier(input_df):
    output_df = input_df

    output_df["dist_pre"] = 0
    output_df["dist_pro"] = 0

    output_df['latDeg_pre'] = output_df['latDeg'].shift(periods=1,fill_value=0)
    output_df['lngDeg_pre'] = output_df['lngDeg'].shift(periods=1,fill_value=0)
    output_df['latDeg_pro'] = output_df['latDeg'].shift(periods=-1,fill_value=0)
    output_df['lngDeg_pro'] = output_df['lngDeg'].shift(periods=-1,fill_value=0)
    output_df['dist_pre'] = calc_haversine(output_df.latDeg_pre, output_df.lngDeg_pre, output_df.latDeg, output_df.lngDeg)
    output_df['dist_pro'] = calc_haversine(output_df.latDeg, output_df.lngDeg, output_df.latDeg_pro, output_df.lngDeg_pro)

    list_phone = output_df['phone'].unique()
    for phone in list_phone:
        ind_s = output_df[test_base['phone'] == phone].index[0]
        ind_e = output_df[test_base['phone'] == phone].index[-1]
        output_df.loc[ind_s,'dist_pre'] = 0
        output_df.loc[ind_e,'dist_pro'] = 0

    pro_95 = output_df['dist_pro'].mean() + (output_df['dist_pro'].std() * 2)
    pre_95 = output_df['dist_pre'].mean() + (output_df['dist_pre'].std() * 2)
    ind = output_df[(output_df['dist_pro'] > pro_95)&(output_df['dist_pre'] > pre_95)][['dist_pre','dist_pro']].index

    for i in ind:
        output_df.loc[i,'latDeg'] = (output_df.loc[i-1,'latDeg'] + output_df.loc[i+1,'latDeg'])/2
        output_df.loc[i,'lngDeg'] = (output_df.loc[i-1,'lngDeg'] + output_df.loc[i+1,'lngDeg'])/2

    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]

    return output_df

##phones mean 

In [ ]:
def add_distance_diff(df):
    df['latDeg_pre'] = df['latDeg'].shift(1)
    df['latDeg_pro'] = df['latDeg'].shift(-1)
    df['lngDeg_pre'] = df['lngDeg'].shift(1)
    df['lngDeg_pro'] = df['lngDeg'].shift(-1)
    df['phone_pre'] = df['phone'].shift(1)
    df['phone_pro'] = df['phone'].shift(-1)
    
    df['dist_pre'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_pre'], df['lngDeg_pre'])
    df['dist_pro'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_pro'], df['lngDeg_pro'])
        
    df.loc[df['phone']!=df['phone_pre'], ['latDeg_pre', 'lngDeg_pre', 'dist_pre']] = np.nan
    df.loc[df['phone']!=df['phone_pro'], ['latDeg_pro', 'lngDeg_pro', 'dist_pro']] = np.nan
        
    return df


def make_lerp_data(input_df):
    org_colus = input_df.columns

    time_list = input_df[["collectionName", "millisSinceGpsEpoch"]].drop_duplicates()
    phone_list = input_df[["collectionName", "phoneName"]].drop_duplicates()
    tmp = time_list.merge(phone_list, on="collectionName", how="outer")

    output_df = tmp.merge(input_df, on=["collectionName", "millisSinceGpsEpoch", "phoneName"], how="left")
    output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
    output_df = output_df.sort_values(["phone", "millisSinceGpsEpoch"])

    output_df["latDeg_pre"] = output_df["latDeg"].shift(1)
    output_df["latDeg_pro"] = output_df["latDeg"].shift(-1)
    output_df["lngDeg_pre"] = output_df["lngDeg"].shift(1)
    output_df["lngDeg_pro"] = output_df["lngDeg"].shift(-1)
    output_df["phone_pre"] = output_df["phone"].shift(1)
    output_df["phone_pro"] = output_df["phone"].shift(-1)
    output_df["millisSinceGpsEpoch_pre"] = output_df["millisSinceGpsEpoch"].shift(1)
    output_df["millisSinceGpsEpoch_pro"] = output_df["millisSinceGpsEpoch"].shift(-1)

    output_df = output_df[(output_df["latDeg"].isnull())&(output_df["phone"] == output_df["phone_pre"])&
                        (output_df["phone"] == output_df["phone_pro"])].copy()

    #preとproの間を経過時間を考慮して算出
    output_df["latDeg"] = output_df["latDeg_pre"] + ((output_df["latDeg_pro"] - output_df["latDeg_pre"]) * 
                                                    ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                    (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
    output_df["lngDeg"] = output_df["lngDeg_pre"] + ((output_df["lngDeg_pro"] - output_df["lngDeg_pre"]) * 
                                                    ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                    (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        
    output_df = output_df[~output_df['latDeg'].isnull()]

    return output_df[org_colus]

def calc_mean_pred(input_df, lerp_df):
    input_df["phone"] = input_df["collectionName"] + "_" + input_df["phoneName"]
    add_lerp = pd.concat([input_df, lerp_df])
    mean_pred_result = add_lerp.groupby(["collectionName", "millisSinceGpsEpoch"])[["latDeg", "lngDeg"]].mean().reset_index()
    output_df = input_df[["collectionName", "phoneName", "millisSinceGpsEpoch"]].copy()
    output_df = output_df.merge(mean_pred_result[["collectionName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]],
                                    on=["collectionName", "millisSinceGpsEpoch"], how="left")
    output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
    return output_df

In [ ]:
def mean_prediction_train(input_df):
    input_df["phone"] = input_df["collectionName"] + "_" + input_df["phoneName"]

    input_df_ = add_distance_diff(input_df)
    th = 43
    input_df_.loc[((input_df_['dist_pre'] > th) & (input_df_['dist_pro'] > th)), ['latDeg', 'lngDeg']] = np.nan
    
    lerp = make_lerp_data(input_df_)
    mean_pred  = calc_mean_pred(input_df_, lerp)

    output_df = kalman_filter(mean_pred)
    output_df["t_latDeg"] = input_df["t_latDeg"].values
    output_df["t_lngDeg"] = input_df["t_lngDeg"].values
    
    return output_df

In [ ]:
def mean_prediction(input_df):
    input_df_ = add_distance_diff(input_df)
    th = 43
    input_df_.loc[((input_df_['dist_pre'] > th) & (input_df_['dist_pro'] > th)), ['latDeg', 'lngDeg']] = np.nan

    test_lerp = make_lerp_data(input_df_)
    test_mean_pred  = calc_mean_pred(input_df_, test_lerp)

    output_df = kalman_filter(test_mean_pred)    
    
    return output_df

##remove device

In [ ]:
def get_removedevice(input_df: pd.DataFrame, divece: str) -> pd.DataFrame:
    input_df['index'] = input_df.index
    input_df = input_df.sort_values('millisSinceGpsEpoch')
    input_df.index = input_df['millisSinceGpsEpoch'].values

    output_df = pd.DataFrame() 
    for _, subdf in input_df.groupby('collectionName'):

        phones = subdf['phoneName'].unique()

        if (len(phones) == 1) or (not divece in phones):
            output_df = pd.concat([output_df, subdf])
            continue

        origin_df = subdf.copy()
            
        _index = subdf['phoneName']==divece
        subdf.loc[_index, 'latDeg'] = np.nan
        subdf.loc[_index, 'lngDeg'] = np.nan
        subdf = subdf.interpolate(method='index', limit_area='inside')

        _index = subdf['latDeg'].isnull()
        subdf.loc[_index, 'latDeg'] = origin_df.loc[_index, 'latDeg'].values
        subdf.loc[_index, 'lngDeg'] = origin_df.loc[_index, 'lngDeg'].values

        output_df = pd.concat([output_df, subdf])

    output_df.index = output_df['index'].values
    output_df = output_df.sort_index()

    del output_df['index']
        
    return output_df

In [151]:
#remove_device
def remove_device_S(input_df):

    output_df = get_removedevice(input_df, 'SamsungS20Ultra')

    return output_df

In [152]:
#remove_device
def remove_device_P(input_df):

    output_df = get_removedevice(input_df, 'Pixel4')

    return output_df

##remove device check score

In [ ]:
def create_dataset_truth_train_for_remove_device(input_df, input_list):    
    output_df = input_df.copy()
    lat_list = []
    lng_list = []
    for collection in input_list:
        collection_df = train_base[train_base["collectionName"] == collection]
        for phone in collection_df["phone"].unique():
            phone_df = collection_df[collection_df["phone"] == phone]
            phone_df["t_latDeg_pro"] = phone_df["t_latDeg"].shift(-1)
            phone_df["t_lngDeg_pro"] = phone_df["t_lngDeg"].shift(-1)
            for lat, lng, lat_pre, lng_pre in zip(
                phone_df["t_latDeg"].to_numpy(),
                phone_df["t_lngDeg"].to_numpy(),
                phone_df["t_latDeg_pro"].to_numpy(),
                phone_df["t_lngDeg_pro"].to_numpy()
            ):
                lat_list.append(lat)
                lng_list.append(lng)
                lat_mean = (lat + lat_pre)/2
                lng_mean = (lng + lng_pre)/2
                lat_list.append(lat_mean)
                lng_list.append(lng_mean)

    points_list = []
    for lat, lng in zip(
        lat_list,
        lng_list
    ):
        points_list.append([lat, lng])

    nearest_points_list = []
    for lat, lng in zip(
        output_df["latDeg"].to_numpy(),
        output_df["lngDeg"].to_numpy()
    ):
        nearest_points_list.append(nearest_point(lat, lng, points_list))

    nearest_points_df = pd.DataFrame(nearest_points_list)
    output_df["latDeg_from_gt"] = nearest_points_df[0].values
    output_df["lngDeg_from_gt"] = nearest_points_df[1].values
    
    return output_df


In [ ]:
def move_closer_truth_for_remove_device(input_df, train_list, test_list):
    test_dfs = []
    for collection in input_df["collectionName"].unique():
        if collection in test_list:
            phones_dfs = []
            collection_df = input_df[input_df["collectionName"] == collection]
            for phone in collection_df["phone"].unique():
                phone_df = collection_df[collection_df["phone"] == phone]
                phone_df = create_dataset_truth_train_for_remove_device(phone_df, train_list)
                phones_dfs.append(phone_df)
            collection_df = pd.concat(phones_dfs)
            test_dfs.append(collection_df)
        else:
            test_dfs.append(input_df[input_df["collectionName"] == collection])
    output_df = pd.concat(test_dfs)

    return output_df

In [ ]:
train_area_1 = ['2020-05-14-US-MTV-1', '2020-05-14-US-MTV-2',
                '2020-05-21-US-MTV-1', '2020-05-21-US-MTV-2',
                '2020-06-04-US-MTV-1', '2020-07-08-US-MTV-1',
                '2020-08-03-US-MTV-1', '2020-09-04-US-SF-1',
                '2021-04-15-US-MTV-1']
test_area_1 = ['2020-05-28-US-MTV-1', '2020-05-28-US-MTV-2',
               '2020-06-04-US-MTV-2', '2020-06-10-US-MTV-1', 
               '2020-06-10-US-MTV-2', '2020-08-03-US-MTV-2']

train_area_2 = ['2021-04-22-US-SJC-1', '2021-04-28-US-SJC-1', 
                '2021-04-29-US-SJC-2']
test_area_2 = ['2021-04-22-US-SJC-2', '2021-04-29-US-SJC-3']

train_area_3 = ['2021-04-28-US-SJC-1', '2021-04-29-US-MTV-1']
test_area_3 = ['2021-03-16-US-RWC-2', '2021-04-21-US-MTV-1',
               '2021-04-28-US-MTV-2', '2021-04-29-US-MTV-2']

train_area_4 = ['2021-03-10-US-SVL-1', '2021-04-26-US-SVL-1']
test_area_4 = ['2021-04-26-US-SVL-2']

train_area_5 = ['2020-06-05-US-MTV-2', '2020-08-06-US-MTV-2', 
                '2020-09-04-US-SF-2', '2021-01-04-US-RWC-1', 
                '2021-01-04-US-RWC-2']
test_area_5 = ['2020-05-15-US-MTV-1', '2020-08-13-US-MTV-1',
               '2021-03-16-US-MTV-2']

In [ ]:
def score_for_remove_device(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.latDeg_from_gt, r.lngDeg_from_gt
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    
    return meter_score

In [ ]:
def get_remove_list(input_df):
    no_train_list = ["2021-03-25-US-PAO-1", "2021-04-02-US-SJC-1", "2021-04-08-US-MTV-1"]
    remove_list = []
    for collection in input_df["collectionName"].unique():
        if collection in no_train_list:
            remove_list.append("SamsungS20Ultra")
        else:
            collection_df = input_df[input_df["collectionName"] == collection]
            phone_list = collection_df["phoneName"].unique()
            score_list = []
            if len(phone_list) == 1:
                remove_list.append("dont_remove")
            else:
                for phone in phone_list:
                    phone_df = collection_df[collection_df["phoneName"] == phone]
                    score = score_for_remove_device(phone_df)
                    score_list.append(score)

                worst_score = sorted(score_list)[-1]
                remove_idx = score_list.index(worst_score)
                print(phone_list[remove_idx])
                remove_list.append(phone_list[remove_idx])
    return remove_list

In [ ]:
def remove_device_by_score(input_df):
    input_df_ = move_closer_truth_for_remove_device(input_df, train_area_1, test_area_1)
    input_df_ = move_closer_truth_for_remove_device(input_df_, train_area_2, test_area_2)
    input_df_ = move_closer_truth_for_remove_device(input_df_, train_area_3, test_area_3)
    input_df_ = move_closer_truth_for_remove_device(input_df_, train_area_4, test_area_4)
    input_df_ = move_closer_truth_for_remove_device(input_df_, train_area_5, test_area_5)
    display(input_df_)
    remove_list = get_remove_list(input_df_)
    dfs = []
    for collection, remove_phone in zip(input_df_["collectionName"].unique(), remove_list):
        collection_df = input_df_[input_df_["collectionName"] == collection]
        if remove_phone == "dont_remove":
            dfs.append(collection_df)
        else:
            removed_df = get_removedevice(collection_df, remove_phone)
            dfs.append(removed_df)
    output_df = pd.concat(dfs)
    return output_df

##position shift

In [ ]:
def position_shift_train(input_df):
    sub_cols = sub.columns

    train_p_s = pd.read_csv(dir / "baseline_locations_train.csv")
    train_b = train_p_s[sub_cols]
    test_b = test_base[sub_cols]

    msge = "millisSinceGpsEpoch"

    testdir = dir / 'test'
    traindir = dir / 'train'

    g_t = pd.DataFrame()
    for d in os.listdir(traindir):
        for p in os.listdir(traindir/d):
            g_t = g_t.append(pd.read_csv(traindir/d/p/'ground_truth.csv'))

    g_t["phone"] = g_t["collectionName"] + "_" + g_t["phoneName"]
    g_t_sub_cols = g_t[sub_cols]

    transformer = pyproj.Transformer.from_crs(
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)
    
    def compute_dist(fname_1, fname_2):
        oof = fname_1
        g_t = fname_2
        df = oof.merge(g_t, on=["phone", "millisSinceGpsEpoch"])
        dist_oof = calc_haversine(df.latDeg_x, df.lngDeg_x, df.latDeg_y, df.lngDeg_y)
        scores = pd.DataFrame({"phone":df.phone, "dist":dist_oof})
        scores_grp = scores.groupby("phone")
        d_50 = scores_grp.quantile(.50).reset_index()
        d_50.columns = ["phone", "q_50"]
        d_95 = scores_grp.quantile(.95).reset_index()
        d_95.columns = ["phone", "q_95"]
        return (scores_grp.quantile(.50).mean() + scores_grp.quantile(.95).mean()) / 2, d_50.merge(d_95)


    def WGS84_to_ECEF(lat, lng, alt):
        rad_lat = lat * (np.pi / 180.0)
        rad_lng = lng * (np.pi / 180.0)
        a = 6378137.0
        finv = 298.257223563
        f = 1 / finv   
        e2 = 1 - (1 - f) * (1 - f)    
        N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
        x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lng)
        y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lng)
        z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
        return x, y, z    


    def ECEF_to_WGS84(x, y, z):
        lng, lat, alt = transformer.transform(x, y, z, radians=False)
        return lng, lat, alt

    
    def position_shift_(input_df, a):
        output_df = input_df.copy()
        output_df["heightAboveWgs84EllipsoidM"] = 63.5
        output_df["x"], output_df["y"], output_df["z"] = zip(
            *output_df.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, 
                                                    x.heightAboveWgs84EllipsoidM), 
                            axis=1)

        )
        output_df.sort_values(["phone", msge], inplace=True)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_p"]] = output_df[fi].shift().where(output_df["phone"].eq(output_df["phone"].shift()))
            output_df[[fi + "_diff"]] = output_df[fi] - output_df[fi + "_p"]

        output_df[["dist"]] = np.sqrt(output_df["x_diff"]**2 + output_df["y_diff"]**2 + output_df["z_diff"]**2)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_new"]] = output_df[fi + "_p"] + output_df[fi + "_diff"] * (1 - a/output_df["dist"])
        lng, lat, alt = ECEF_to_WGS84(output_df["x_new"].values, output_df["y_new"].values, output_df["z_new"].values)

        lat[np.isnan(lat)] = output_df.loc[np.isnan(lat), "latDeg"]
        lng[np.isnan(lng)] = output_df.loc[np.isnan(lng), "lngDeg"]
        output_df["latDeg"] = lat
        output_df["lngDeg"] = lng

        output_df.sort_values(["phone", msge], inplace=True)

        return output_df[sub_cols]

    
    def objective(trial):
        a = trial.suggest_uniform("a", -1, 1)
        score, scores = compute_dist(position_shift_(train_b, a), g_t)
        return score

    study = optuna.create_study()
    study.optimize(objective, n_trials=30)

    output_df = position_shift_(input_df, a=study.best_params["a"])

    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]
    output_df["t_latDeg"] = input_df["t_latDeg"]
    output_df["t_lngDeg"] = input_df["t_lngDeg"]

    return output_df

In [ ]:
def position_shift(input_df):
    sub_cols = sub.columns

    train_p_s = pd.read_csv(dir / "baseline_locations_train.csv")
    train_b = train_p_s[sub_cols]
    test_b = test_base[sub_cols]

    msge = "millisSinceGpsEpoch"

    testdir = dir / 'test'
    traindir = dir / 'train'

    g_t = pd.DataFrame()
    for d in os.listdir(traindir):
        for p in os.listdir(traindir/d):
            g_t = g_t.append(pd.read_csv(traindir/d/p/'ground_truth.csv'))

    g_t["phone"] = g_t["collectionName"] + "_" + g_t["phoneName"]
    g_t_sub_cols = g_t[sub_cols]

    transformer = pyproj.Transformer.from_crs(
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)
    
    def compute_dist(fname_1, fname_2):
        oof = fname_1
        g_t = fname_2
        df = oof.merge(g_t, on=["phone", "millisSinceGpsEpoch"])
        dist_oof = calc_haversine(df.latDeg_x, df.lngDeg_x, df.latDeg_y, df.lngDeg_y)
        scores = pd.DataFrame({"phone":df.phone, "dist":dist_oof})
        scores_grp = scores.groupby("phone")
        d_50 = scores_grp.quantile(.50).reset_index()
        d_50.columns = ["phone", "q_50"]
        d_95 = scores_grp.quantile(.95).reset_index()
        d_95.columns = ["phone", "q_95"]
        return (scores_grp.quantile(.50).mean() + scores_grp.quantile(.95).mean()) / 2, d_50.merge(d_95)


    def WGS84_to_ECEF(lat, lng, alt):
        rad_lat = lat * (np.pi / 180.0)
        rad_lng = lng * (np.pi / 180.0)
        a = 6378137.0
        finv = 298.257223563
        f = 1 / finv   
        e2 = 1 - (1 - f) * (1 - f)    
        N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
        x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lng)
        y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lng)
        z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
        return x, y, z    


    def ECEF_to_WGS84(x, y, z):
        lng, lat, alt = transformer.transform(x, y, z, radians=False)
        return lng, lat, alt

    
    def position_shift_(input_df, a):
        output_df = input_df.copy()
        output_df["heightAboveWgs84EllipsoidM"] = 63.5
        output_df["x"], output_df["y"], output_df["z"] = zip(
            *output_df.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, 
                                                    x.heightAboveWgs84EllipsoidM), 
                            axis=1)

        )
        output_df.sort_values(["phone", msge], inplace=True)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_p"]] = output_df[fi].shift().where(output_df["phone"].eq(output_df["phone"].shift()))
            output_df[[fi + "_diff"]] = output_df[fi] - output_df[fi + "_p"]

        output_df[["dist"]] = np.sqrt(output_df["x_diff"]**2 + output_df["y_diff"]**2 + output_df["z_diff"]**2)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_new"]] = output_df[fi + "_p"] + output_df[fi + "_diff"] * (1 - a/output_df["dist"])
        lng, lat, alt = ECEF_to_WGS84(output_df["x_new"].values, output_df["y_new"].values, output_df["z_new"].values)

        lat[np.isnan(lat)] = output_df.loc[np.isnan(lat), "latDeg"]
        lng[np.isnan(lng)] = output_df.loc[np.isnan(lng), "lngDeg"]
        output_df["latDeg"] = lat
        output_df["lngDeg"] = lng

        output_df.sort_values(["phone", msge], inplace=True)

        return output_df[sub_cols]

    
    def objective(trial):
        a = trial.suggest_uniform("a", -1, 1)
        score, scores = compute_dist(position_shift_(train_b, a), g_t)
        return score

    study = optuna.create_study()
    study.optimize(objective, n_trials=30)

    output_df = position_shift_(input_df, a=study.best_params["a"])

    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]

    return output_df

##adaptive_gauss+phone_mean

In [ ]:
def apply_gauss_smoothing(df, params):
    SZ_1 = params['sz_1']
    SZ_2 = params['sz_2']
    SZ_CRIT = params['sz_crit']    
    
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in unique_paths:
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].to_numpy()
                
        lat_g1 = gaussian_filter1d(data[:, 0], np.sqrt(SZ_1))
        lon_g1 = gaussian_filter1d(data[:, 1], np.sqrt(SZ_1))
        lat_g2 = gaussian_filter1d(data[:, 0], np.sqrt(SZ_2))
        lon_g2 = gaussian_filter1d(data[:, 1], np.sqrt(SZ_2))

        lat_dif = data[1:,0] - data[:-1,0]
        lon_dif = data[1:,1] - data[:-1,1]

        lat_crit = np.append(np.abs(gaussian_filter1d(lat_dif, np.sqrt(SZ_CRIT)) / (1e-9 + gaussian_filter1d(np.abs(lat_dif), np.sqrt(SZ_CRIT)))),[0])
        lon_crit = np.append(np.abs(gaussian_filter1d(lon_dif, np.sqrt(SZ_CRIT)) / (1e-9 + gaussian_filter1d(np.abs(lon_dif), np.sqrt(SZ_CRIT)))),[0])           
            
        df.loc[cond, 'latDeg'] = lat_g1 * lat_crit + lat_g2 * (1.0 - lat_crit)
        df.loc[cond, 'lngDeg'] = lon_g1 * lon_crit + lon_g2 * (1.0 - lon_crit)    
                       
    return df

In [ ]:
def mean_with_other_phones(df):
    collections_list = df[['collectionName']].drop_duplicates().to_numpy()

    for collection in collections_list:
        phone_list = df[df['collectionName'].to_list() == collection][['phoneName']].drop_duplicates().to_numpy()

        phone_data = {}
        corrections = {}
        for phone in phone_list:
            cond = np.logical_and(df['collectionName'] == collection[0], df['phoneName'] == phone[0]).to_list()
            phone_data[phone[0]] = df[cond][['millisSinceGpsEpoch', 'latDeg', 'lngDeg']].to_numpy()

        for current in phone_data:
            correction = np.ones(phone_data[current].shape, dtype=np.float)
            correction[:,1:] = phone_data[current][:,1:]
            
            # Telephones data don't complitely match by time, so - interpolate.
            for other in phone_data:
                if other == current:
                    continue

                loc = interp1d(phone_data[other][:,0], 
                               phone_data[other][:,1:], 
                               axis=0, 
                               kind='linear', 
                               copy=False, 
                               bounds_error=None, 
                               fill_value='extrapolate', 
                               assume_sorted=True)
                
                start_idx = 0
                stop_idx = 0
                for idx, val in enumerate(phone_data[current][:,0]):
                    if val < phone_data[other][0,0]:
                        start_idx = idx
                    if val < phone_data[other][-1,0]:
                        stop_idx = idx

                if stop_idx - start_idx > 0:
                    correction[start_idx:stop_idx,0] += 1
                    correction[start_idx:stop_idx,1:] += loc(phone_data[current][start_idx:stop_idx,0])                    

            correction[:,1] /= correction[:,0]
            correction[:,2] /= correction[:,0]
            
            corrections[current] = correction.copy()
        
        for phone in phone_list:
            cond = np.logical_and(df['collectionName'] == collection[0], df['phoneName'] == phone[0]).to_list()
            
            df.loc[cond, ['latDeg', 'lngDeg']] = corrections[phone[0]][:,1:]            
            
    return df

In [ ]:
def gauss_filter(input_df):
    input_df_ = input_df.copy()
    output_df = mean_with_other_phones(
        apply_gauss_smoothing(input_df_, {'sz_1' : 0.85, 'sz_2' : 5.65, 'sz_crit' : 1.5})
    )
    return output_df

##kalman filter

In [ ]:
def apply_kf_smoothing(input_df, kf_):
    output_df = input_df.copy()
    unique_paths = output_df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in tqdm(unique_paths):
        cond = np.logical_and(output_df['collectionName'] == collection, output_df['phoneName'] == phone)
        data = output_df[cond][['latDeg', 'lngDeg']].to_numpy()
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        output_df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
        output_df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
    return output_df


In [ ]:
def kalman_filter(input_df):
    T = 1.0
    state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                                [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
    process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
    observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
    observation_noise = np.diag([3e-5, 3e-5]) + np.ones((2, 2)) * 1e-9

    kf = simdkalman.KalmanFilter(
            state_transition = state_transition,
            process_noise = process_noise,
            observation_model = observation_model,
            observation_noise = observation_noise)
    
    output_df = apply_kf_smoothing(input_df, kf)

    return output_df


##move closer center SJC

In [ ]:
def nearest_point(x, y, points):
    result = {}
    if len(points) == 0:
        return result
    result[0] = points[0][0]
    result[1] = points[0][1]
    stdval = math.sqrt((points[0][0] - x) ** 2 + (points[0][1] - y) ** 2)
    for point in points:
        distance = math.sqrt((point[0] - x) ** 2 + (point[1] - y) ** 2)
        if stdval > distance:
            result[0] = point[0]
            result[1] = point[1]
            stdval = distance
    return [result[0], result[1]]

In [ ]:
def create_dataset_center(input_df):
    output_df = input_df.copy()
    collection_list = output_df["collectionName"].unique()

    line_points_dfs = []
    c = 1
    for collection in collection_list:
        print(f"now : {c}/{len(collection_list)}")
        c += 1
        print(collection)
        target_df = output_df[output_df["collectionName"] == collection].reset_index(drop=True)
        target_df["geometry"] = [Point(p) for p in target_df[["lngDeg", "latDeg"]].to_numpy()]
        target_gdf = gpd.GeoDataFrame(target_df, geometry=target_df["geometry"])

        offset = 0.1**5
        bbox = target_gdf.bounds + [-(offset+0.01), -(offset+0.01), offset+0.01, offset+0.01]
        east = bbox["minx"].min()
        west = bbox["maxx"].max()
        south = bbox["miny"].min()
        north = bbox["maxy"].max()
        G = ox.graph.graph_from_bbox(north, south, east, west, network_type='drive')

        nodes, edges = momepy.nx_to_gdf(G)
        
        edges = edges.dropna(subset=["geometry"]).reset_index(drop=True)
        hits = bbox.apply(lambda row: list(edges.sindex.intersection(row)), axis=1)
        tmp = pd.DataFrame({
            "pt_idx": np.repeat(hits.index, hits.apply(len)),
            "line_i": np.concatenate(hits.values)
        })
        tmp = tmp.join(edges.reset_index(drop=True), on="line_i")
        tmp = tmp.join(target_gdf.geometry.rename("point"), on="pt_idx")
        tmp = gpd.GeoDataFrame(tmp, geometry="geometry", crs=target_gdf.crs)

        tmp["snap_dist"] = tmp.geometry.distance(gpd.GeoSeries(tmp.point))

        tolerance = 0.0005  
        tmp = tmp.loc[tmp.snap_dist <= tolerance]
        tmp = tmp.sort_values(by=["snap_dist"])

        closest = tmp.groupby("pt_idx").first()
        closest = gpd.GeoDataFrame(closest, geometry="geometry")
        closest = closest.drop_duplicates("line_i").reset_index(drop=True)

        line_points_list = []
        split = 200  # param: number of split in each LineString
        for dist in range(0, split, 1):
            dist = dist/split
            line_points = closest["geometry"].interpolate(dist, normalized=True)
            line_points_list.append(line_points)
        line_points = pd.concat(line_points_list).reset_index(drop=True)
        line_points = line_points.reset_index().rename(columns={0:"geometry"})
        line_points["lngDeg"] = line_points["geometry"].x
        line_points["latDeg"] = line_points["geometry"].y
        
        line_points_ = line_points.loc[:, ["lngDeg", "latDeg"]]
        line_points_dfs.append(line_points_)

    line_points_df = pd.concat(line_points_dfs)
    line_points_list = sorted(line_points_df.values.tolist())
    
    print(len(line_points_list))

    nearest_point_list = []
    
    for lng, lat in zip(
        output_df["lngDeg"].to_numpy(),
        output_df["latDeg"].to_numpy()
    ):
        nearest_point_list.append(nearest_point(lng, lat, line_points_list))
        
    nearest_point_df = pd.DataFrame(nearest_point_list)
    output_df["latDeg_center"] = nearest_point_df[1].values
    output_df["lngDeg_center"] = nearest_point_df[0].values
    
    
        
    output_df["latDeg_center_pre"] = output_df["latDeg_center"].shift(1)
    output_df["lngDeg_center_pre"] = output_df["lngDeg_center"].shift(1)

    output_df['meter'] = output_df.apply(
            lambda r: calc_haversine(
                r.latDeg, r.lngDeg, r.latDeg_center, r.lngDeg_center
            ),
            axis=1
        )
    
    rl_list = []
    #for collection in output_df["collectionName"].unique():
        #for phone in output_df["phoneName"].unique():
            #phone_df = output_df[output_df["phone"] == f"{collection}_{phone}"]
    for lat, lng, lat_c, lng_c, lat_c_pre, lng_c_pre in zip(
                output_df["latDeg"].to_numpy(),
                output_df["lngDeg"].to_numpy(),
                output_df["latDeg_center"].to_numpy(),
                output_df["lngDeg_center"].to_numpy(),
                output_df["latDeg_center_pre"].to_numpy(),
                output_df["lngDeg_center_pre"].to_numpy()
            ):
                base_vec = np.array([lat-lat_c_pre, lng-lng_c_pre])
                center_vec = np.array([lat_c-lat_c_pre, lng_c-lng_c_pre])
                r_or_l = np.cross(base_vec,center_vec)
                if r_or_l > 0:
                    rl_list.append("Right")
                else:
                    rl_list.append("Left")
    output_df["Right_or_left"] = rl_list
    
    return output_df

In [ ]:
def check_meter_from_center(input_df):
        output_df = input_df.copy()
        lat_list = []
        lng_list = []
        for lat, lng, lat_c, lng_c, meter, rl in zip(
            output_df["latDeg"].to_numpy(),
            output_df["lngDeg"].to_numpy(),
            output_df["latDeg_center"].to_numpy(),
            output_df["lngDeg_center"].to_numpy(),
            output_df["meter"].to_numpy(),
            output_df["Right_or_left"].values
        ):
            if rl == "Left":
                lat = lat_c
                lng = lng_c
                lat_list.append(lat)
                lng_list.append(lng)
            else:
                if meter > 30:
                    lat = lat
                    lng = lng
                elif meter > 10:
                    lat = ((((lat + lat_c)/2 + lat_c)/2 + lat_c)/2 + lat_c)/2
                    lng = ((((lng + lng_c)/2 + lng_c)/2 + lng_c)/2 + lng_c)/2
                elif meter > 7.5:
                    lat = (((lat + lat_c)/2 + lat_c)/2 + lat_c)/2
                    lng = (((lng + lng_c)/2 + lng_c)/2 + lng_c)/2
                elif meter > 5:
                    lat = ((lat + lat_c)/2 + lat_c)/2
                    lng = ((lng + lng_c)/2 + lng_c)/2
                elif meter > 2.5:
                    lat = (lat + lat_c)/2
                    lng = (lng + lng_c)/2

                lat_list.append(lat)
                lng_list.append(lng)


        output_df.iloc[:, 2] = lat_list
        output_df.iloc[:, 3] = lng_list
        return output_df

In [ ]:
def move_closer_center_train(input_df):
    train_SJC_list = ["2021-04-22-US-SJC-1", "2021-04-28-US-SJC-1", "2021-04-29-US-SJC-2"]
    train_dfs = []
    for collection in input_df["collectionName"].unique():
        if collection in train_SJC_list:
            phones_dfs = []
            collection_df = input_df[input_df["collectionName"] == collection]
            for phone in collection_df["phone"].unique():
                phone_df = collection_df[collection_df["phone"] == phone]
                phone_df = create_dataset_center(phone_df)
                phone_df = check_meter_from_center(phone_df)
                phones_dfs.append(phone_df)
            collection_df = pd.concat(phones_dfs)
            collection_df = mean_points(collection_df)
            train_dfs.append(collection_df)
        else:
            train_dfs.append(input_df[input_df["collectionName"] == collection])
    output_df = pd.concat(train_dfs)

    return output_df

In [ ]:
def move_closer_center(input_df):
    test_SJC_list = ["2021-04-22-US-SJC-2", "2021-04-29-US-SJC-3"]
    test_dfs = []
    for collection in input_df["collectionName"].unique():
        if collection in test_SJC_list:
            phones_dfs = []
            collection_df = input_df[input_df["collectionName"] == collection]
            for phone in collection_df["phone"].unique():
                phone_df = collection_df[collection_df["phone"] == phone]
                phone_df = create_dataset_center(phone_df)
                phone_df = check_meter_from_center(phone_df)
                phones_dfs.append(phone_df)
            collection_df = pd.concat(phones_dfs)
            collection_df = mean_points(collection_df)
            test_dfs.append(collection_df)
        else:
            test_dfs.append(input_df[input_df["collectionName"] == collection])
    output_df = pd.concat(test_dfs)

    return output_df

##move closer truth SJC

In [ ]:
def create_dataset_truth_train(input_df):
    train_SJC_list = ["2021-04-22-US-SJC-1", "2021-04-28-US-SJC-1", "2021-04-29-US-SJC-2"]
    #train_SJCs = []
    #for SJC in train_SJC_list:
       # SJC_df = input_df[input_df["collectionName"] == SJC]
       # train_SJCs.append(SJC_df)
    #train_SJC = pd.concat(train_SJCs)
    output_df = input_df.copy()
    lat_list = []
    lng_list = []
    for collection in train_SJC_list:
        collection_df = train_base[train_base["collectionName"] == collection]
        for phone in collection_df["phone"].unique():
            phone_df = collection_df[collection_df["phone"] == phone]
            phone_df["t_latDeg_pro"] = phone_df["t_latDeg"].shift(-1)
            phone_df["t_lngDeg_pro"] = phone_df["t_lngDeg"].shift(-1)
            for lat, lng, lat_pre, lng_pre in zip(
                phone_df["t_latDeg"].to_numpy(),
                phone_df["t_lngDeg"].to_numpy(),
                phone_df["t_latDeg_pro"].to_numpy(),
                phone_df["t_lngDeg_pro"].to_numpy()
            ):
                lat_list.append(lat)
                lng_list.append(lng)
                lat_mean = (lat + lat_pre)/2
                lng_mean = (lng + lng_pre)/2
                lat_list.append(lat_mean)
                lng_list.append(lng_mean)

    points_list = []
    for lat, lng in zip(
        lat_list,
        lng_list
    ):
        points_list.append([lat, lng])

    nearest_points_list = []
    for lat, lng in zip(
        output_df["latDeg"].to_numpy(),
        output_df["lngDeg"].to_numpy()
    ):
        nearest_points_list.append(nearest_point(lat, lng, points_list))

    nearest_points_df = pd.DataFrame(nearest_points_list)
    output_df["latDeg_from_gt"] = nearest_points_df[0].values
    output_df["lngDeg_from_gt"] = nearest_points_df[1].values
    
    lat_gt_pre_list = []
    lng_gt_pre_list = []
    for collection in output_df["collectionName"].unique():
        collection_df = output_df[output_df["collectionName"] == collection]
        for phone in collection_df["phoneName"].unique():
            phone_df = collection_df[collection_df["phoneName"] == phone]
            phone_df["latDeg_from_gt_pre"] = phone_df["latDeg_from_gt"].shift(-1)
            phone_df["lngDeg_from_gt_pre"] = phone_df["lngDeg_from_gt"].shift(-1)
            for lat_gt_pre, lng_gt_pre in zip(
                phone_df["latDeg_from_gt_pre"].to_numpy(),
                phone_df["lngDeg_from_gt_pre"].to_numpy(),
            ):
                lat_gt_pre_list.append(lat_gt_pre)
                lng_gt_pre_list.append(lng_gt_pre)    
        
        output_df["latDeg_from_gt_pre"] = lat_gt_pre_list
        output_df["lngDeg_from_gt_pre"] = lng_gt_pre_list

    output_df["meter"] = output_df.apply(
        lambda r:calc_haversine(
            r.latDeg, r.lngDeg, r.latDeg_from_gt, r.lngDeg_from_gt
        ),
        axis=1
    )
    return output_df


In [ ]:
def check_meter_from_truth(input_df):
    output_df = input_df.copy()
    lat_list = []
    lng_list = []
    for lat, lng, lat_gt, lng_gt, meter in zip(
        output_df["latDeg"].to_numpy(),
        output_df["lngDeg"].to_numpy(),
        output_df["latDeg_from_gt"].to_numpy(),
        output_df["lngDeg_from_gt"].to_numpy(),
        output_df["meter"].to_numpy(),
    ):
        if meter > 30:
            lat = lat
            lng = lng
        elif meter > 10:
            lat = ((((lat + lat_gt)/2 + lat_gt)/2 + lat_gt)/2 + lat_gt)/2
            lng = ((((lng + lng_gt)/2 + lng_gt)/2 + lng_gt)/2 + lng_gt)/2
        elif meter > 7.5:
            lat = (((lat + lat_gt)/2 + lat_gt)/2 + lat_gt)/2
            lng = (((lng + lng_gt)/2 + lng_gt)/2 + lng_gt)/2
        elif meter > 5:
            lat = ((lat + lat_gt)/2 + lat_gt)/2
            lng = ((lng + lng_gt)/2 + lng_gt)/2
        elif meter > 2.5:
            lat = (lat + lat_gt)/2
            lng = (lng + lng_gt)/2
        elif meter > 1:
            lat = ((lat + lat_gt)/2 + lat)/2
            lng = ((lng + lng_gt)/2 + lng)/2
        else:
            lat = lat_gt
            lng = lng_gt

        lat_list.append(lat)
        lng_list.append(lng)


    output_df.iloc[:, 2] = lat_list
    output_df.iloc[:, 3] = lng_list
    return output_df

In [ ]:
def move_closer_truth_train(input_df):
    train_SJC_list = ["2021-04-22-US-SJC-1", "2021-04-28-US-SJC-1", "2021-04-29-US-SJC-2"]
    train_dfs = []
    for collection in input_df["collectionName"].unique():
        if collection in train_SJC_list:
            phones_dfs = []
            collection_df = input_df[input_df["collectionName"] == collection]
            for phone in collection_df["phone"].unique():
                phone_df = collection_df[collection_df["phone"] == phone]
                phone_df = create_dataset_truth_train(phone_df)
                phone_df = check_meter_from_truth(phone_df)
                phones_dfs.append(phone_df)
            collection_df = pd.concat(phones_dfs)
            train_dfs.append(collection_df)
        else:
            train_dfs.append(input_df[input_df["collectionName"] == collection])
    output_df = pd.concat(train_dfs)

    return output_df

In [ ]:
def move_closer_truth(input_df):
    test_SJC_list = ["2021-04-22-US-SJC-2", "2021-04-29-US-SJC-3"]
    test_dfs = []
    for collection in input_df["collectionName"].unique():
        if collection in test_SJC_list:
            phones_dfs = []
            collection_df = input_df[input_df["collectionName"] == collection]
            for phone in collection_df["phone"].unique():
                phone_df = collection_df[collection_df["phone"] == phone]
                phone_df = create_dataset_truth_train(phone_df)
                phone_df = check_meter_from_truth(phone_df)
                phones_dfs.append(phone_df)
            collection_df = pd.concat(phones_dfs)
            test_dfs.append(collection_df)
        else:
            test_dfs.append(input_df[input_df["collectionName"] == collection])
    output_df = pd.concat(test_dfs)

    return output_df

#cheke CV

In [ ]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    return output_df

In [156]:
train = train_base.copy()

In [157]:
train = remove_device_P(train)
print('<<<remove_device>>>')
train = check_score(train)

<<<remove_device>>>
error meter: 3.7152727314957783
score: 5.151723902761146


In [158]:
train = remove_device_S(train)
print('<<<remove_device>>>')
train = check_score(train)

<<<remove_device>>>
error meter: 3.6136466267233804
score: 4.997452081955869


In [143]:
train = outlier_train(train)
print('<<<outlier>>>')
train = check_score(train)

<<<outlier>>>
error meter: 3.6776959126304978
score: 5.2890161163079314


In [144]:
train = mean_prediction_train(train)
print('<<<phones_mean>>>')
train = check_score(train)


<<<phones_mean>>>
error meter: 2.909066992535581
score: 4.112424441013549


In [145]:
train = remove_device(train)
print('<<<remove_device>>>')
train = check_score(train)

<<<remove_device>>>
error meter: 2.8217720062789717
score: 3.9666551667884566


In [146]:
train = position_shift_train(train)
print('<<<position_shift>>>')
train = check_score(train)

[I 2021-07-28 10:30:52,497] A new study created in memory with name: no-name-89213d86-e974-411a-a996-a039a9764141
[I 2021-07-28 10:30:59,350] Trial 0 finished with value: 5.199628174408762 and parameters: {'a': 0.8726115433624375}. Best is trial 0 with value: 5.199628174408762.
[I 2021-07-28 10:31:06,104] Trial 1 finished with value: 5.709029713248592 and parameters: {'a': -0.9231521719414253}. Best is trial 0 with value: 5.199628174408762.
[I 2021-07-28 10:31:12,637] Trial 2 finished with value: 5.253543619669742 and parameters: {'a': 0.11619003625999147}. Best is trial 0 with value: 5.199628174408762.
[I 2021-07-28 10:31:19,327] Trial 3 finished with value: 5.518712132941019 and parameters: {'a': -0.576526671180895}. Best is trial 0 with value: 5.199628174408762.
[I 2021-07-28 10:31:26,189] Trial 4 finished with value: 5.253541459867117 and parameters: {'a': 0.11619739955398445}. Best is trial 0 with value: 5.199628174408762.
[I 2021-07-28 10:31:32,989] Trial 5 finished with value: 5

<<<position_shift>>>
error meter: 2.781806632373344
score: 3.9268456551068516


In [148]:
train = remove_device(train)
print('<<<remove_device>>>')
train = check_score(train)

<<<remove_device>>>
error meter: 2.7306584572092234
score: 3.8487913766606066


In [149]:
train = gauss_filter(train)
print('<<<guass_filter>>>')
train = check_score(train)

<<<guass_filter>>>
error meter: 2.586949185563533
score: 3.649056948171927


In [126]:
train = gauss_filter(train)
print('<<<guass_filter>>>')
train = check_score(train)

<<<guass_filter>>>
error meter: 2.6720635449884482
score: 3.6899179856193998


In [159]:
#make triangle 1
CV1 = make_triangle(train_base)
for i in range(100):
    CV1 = make_triangle(CV1)
print('<<<triangle_1>>>')
CV1 = check_score(CV1)
print('--------------------')

#outlier
CV2 = outlier_train(CV1)
print('<<<outlier>>>')
CV2 = check_score(CV2)
print('--------------------')

#make triangle 2
CV3 = make_triangle(CV2)
for i in range(100):
    CV3 = make_triangle(CV3)
print('<<<triangle_2>>>')
CV3 = check_score(CV3)
print('--------------------')

#phonse mean
CV4 = mean_prediction_train(CV3)
print('<<<phones_mean>>>')
CV4 = check_score(CV4)
print('--------------------')

#make triangle 3
CV5 = make_triangle(CV4)
for i in range(100):
    CV5 = make_triangle(CV5)
print('<<<triangle_3>>>')
CV5 = check_score(CV5)
print('--------------------')

#remove device
CV6 = remove_device_P(CV5)
print('<<<remove_device_P>>>')
CV6 = check_score(CV6)
print('--------------------')

#make triangle 4
CV7 = make_triangle(CV6)
for i in range(100):
    CV7 = make_triangle(CV7)
print('<<<triangle_4>>>')
CV7 = check_score(CV7)
print('--------------------')

#remove device
CV8 = remove_device_S(CV7)
print('<<<remove_device_S>>>')
CV8 = check_score(CV8)
print('--------------------')

#make triangle 5
CV9 = make_triangle(CV8)
for i in range(100):
    CV9 = make_triangle(CV9)
print('<<<triangle_4>>>')
CV9 = check_score(CV9)
print('--------------------')

#position shift
CV10 = position_shift_train(CV9)
print('<<<position_shift>>>')
CV10 = check_score(CV10)
print('--------------------')

#make triangle 5
CV11 = make_triangle(CV10)
for i in range(100):
    CV11 = make_triangle(CV11)
print('<<<triangle_5>>>')
CV11 = check_score(CV11)
print('--------------------')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



<<<triangle_1>>>
error meter: 3.3579582394393928
score: 4.753446293953488
--------------------
<<<outlier>>>
error meter: 3.358838280191992
score: 4.753446293953488
--------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars



<<<triangle_2>>>
error meter: 3.345099090946541
score: 4.74999834268959
--------------------



<<<phones_mean>>>
error meter: 2.9007008123380005
score: 3.981980123726002
--------------------


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



<<<triangle_4>>>
error meter: 2.8435099137095623
score: 3.887193101566163
--------------------
<<<remove_device>>>
error meter: 2.75665225335587
score: 3.749463760886066
--------------------
<<<triangle_4>>>
error meter: 2.7541752051500863
score: 3.7450809780751424
--------------------


[I 2021-07-28 11:28:05,200] A new study created in memory with name: no-name-d6413cc5-b2a0-4b4f-abf8-8d2bbf129081
[I 2021-07-28 11:28:11,935] Trial 0 finished with value: 5.737657553415794 and parameters: {'a': -0.972633111657059}. Best is trial 0 with value: 5.737657553415794.
[I 2021-07-28 11:28:18,399] Trial 1 finished with value: 5.327393232709673 and parameters: {'a': -0.12744462862912753}. Best is trial 1 with value: 5.327393232709673.
[I 2021-07-28 11:28:25,447] Trial 2 finished with value: 5.745493034927236 and parameters: {'a': -0.9862401251957731}. Best is trial 1 with value: 5.327393232709673.
[I 2021-07-28 11:28:32,015] Trial 3 finished with value: 5.2601972451538135 and parameters: {'a': 0.09239045266790358}. Best is trial 3 with value: 5.2601972451538135.
[I 2021-07-28 11:28:38,401] Trial 4 finished with value: 5.310905279029827 and parameters: {'a': -0.07914422216264838}. Best is trial 3 with value: 5.2601972451538135.
[I 2021-07-28 11:28:45,207] Trial 5 finished with va

<<<position_shift>>>
error meter: 2.7046342353213446
score: 3.6865237794773895
--------------------


KeyboardInterrupt: ignored

In [160]:
#make triangle 1
CV1 = make_triangle(train_base)
for i in range(100):
    CV1 = make_triangle(CV1)
print('<<<triangle_1>>>')
CV1 = check_score(CV1)
print('--------------------')

#outlier
CV2 = outlier_train(CV1)
print('<<<outlier>>>')
CV2 = check_score(CV2)
print('--------------------')

#make triangle 2
CV3 = make_triangle(CV2)
for i in range(100):
    CV3 = make_triangle(CV3)
print('<<<triangle_2>>>')
CV3 = check_score(CV3)
print('--------------------')

#phonse mean
CV4 = mean_prediction_train(CV3)
print('<<<phones_mean>>>')
CV4 = check_score(CV4)
print('--------------------')

#make triangle 3
CV5 = make_triangle(CV4)
for i in range(100):
    CV5 = make_triangle(CV5)
print('<<<triangle_3>>>')
CV5 = check_score(CV5)
print('--------------------')

#remove device
CV6 = remove_device_P(CV5)
print('<<<remove_device_P>>>')
CV6 = check_score(CV6)
print('--------------------')

#make triangle 4
CV7 = make_triangle(CV6)
for i in range(100):
    CV7 = make_triangle(CV7)
print('<<<triangle_4>>>')
CV7 = check_score(CV7)
print('--------------------')

#remove device
CV8 = remove_device_S(CV7)
print('<<<remove_device_S>>>')
CV8 = check_score(CV8)
print('--------------------')

#make triangle 5
CV9 = make_triangle(CV8)
for i in range(100):
    CV9 = make_triangle(CV9)
print('<<<triangle_4>>>')
CV9 = check_score(CV9)
print('--------------------')

#position shift
CV10 = position_shift_train(CV9)
print('<<<position_shift>>>')
CV10 = check_score(CV10)
print('--------------------')

#make triangle 5
CV11 = make_triangle(CV10)
for i in range(100):
    CV11 = make_triangle(CV11)
print('<<<triangle_5>>>')
CV11 = check_score(CV11)
print('--------------------')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



<<<triangle_1>>>
error meter: 3.3579582394393928
score: 4.753446293953488
--------------------
<<<outlier>>>
error meter: 3.358838280191992
score: 4.753446293953488
--------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



<<<triangle_2>>>
error meter: 3.347407087156006
score: 4.74999834268959
--------------------



<<<phones_mean>>>
error meter: 2.8901166653174113
score: 3.966876017017269
--------------------


...

<<<triangle_3>>>
error meter: 2.8798260020244313
score: 3.95138174468124
--------------------
<<<remove_device_P>>>
error meter: 2.8392545641355205
score: 3.885883332089797
--------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



<<<triangle_4>>>
error meter: 2.835598284272725
score: 3.8755222022961644
--------------------
<<<remove_device_S>>>
error meter: 2.7484448065457534
score: 3.734451383531354
--------------------
<<<triangle_4>>>
error meter: 2.7467062418931016
score: 3.733040426721129
--------------------


[I 2021-07-28 12:16:31,843] A new study created in memory with name: no-name-d2b7cfc4-10f9-411e-abf2-44dac874434f
[I 2021-07-28 12:16:38,285] Trial 0 finished with value: 5.413639279309459 and parameters: {'a': -0.3514811231243091}. Best is trial 0 with value: 5.413639279309459.
[I 2021-07-28 12:16:44,974] Trial 1 finished with value: 5.193087958600306 and parameters: {'a': 0.4794876730672759}. Best is trial 1 with value: 5.193087958600306.
[I 2021-07-28 12:16:51,429] Trial 2 finished with value: 5.5490631622033195 and parameters: {'a': -0.6335965186044061}. Best is trial 1 with value: 5.193087958600306.
[I 2021-07-28 12:16:57,902] Trial 3 finished with value: 5.207497015410404 and parameters: {'a': 0.9413268225011708}. Best is trial 1 with value: 5.193087958600306.
[I 2021-07-28 12:17:04,483] Trial 4 finished with value: 5.306837412126571 and parameters: {'a': -0.06476614107038392}. Best is trial 1 with value: 5.193087958600306.
[I 2021-07-28 12:17:11,024] Trial 5 finished with value:

<<<position_shift>>>
error meter: 2.697366456245622
score: 3.6766336377767455
--------------------
<<<triangle_5>>>
error meter: 2.6913966719246796
score: 3.6626107117912534
--------------------


In [174]:
#guass filter
CV16 = gauss_filter(CV11)
print('<<<guass_filter>>>')
CV16 = check_score(CV16)
print('--------------------')

<<<guass_filter>>>
error meter: 2.5377395296453957
score: 3.5331134392091474
--------------------


In [175]:
CV12 = move_closer_center_train(CV16)
CV12 = check_score(CV12)

now : 1/1
2021-04-22-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




23600
now : 1/1
2021-04-22-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




23000


...

22600
now : 1/1
2021-04-29-US-SJC-2


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




23200


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning:

invalid value encountered in true_divide



error meter: 2.556212928813154
score: 3.5340940540125936


In [176]:
#make triangle 5
CV13 = make_triangle(CV12)
for i in range(100):
    CV13 = make_triangle(CV13)
print('<<<triangle_5>>>')
CV13 = check_score(CV13)
print('--------------------')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



<<<triangle_5>>>
error meter: 2.5557896508756817
score: 3.568196336913206
--------------------


In [177]:
CV14 = move_closer_truth_train(CV12)
CV14 = check_score(CV14)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



error meter: 2.5239127485109476
score: 3.435185170649873


In [169]:
CV12 = move_closer_center_train(CV11)
CV12 = check_score(CV12)

now : 1/1
2021-04-22-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




23800
now : 1/1
2021-04-22-US-SJC-1


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




23800


...

22600
now : 1/1
2021-04-29-US-SJC-2


/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




22800


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning:

invalid value encountered in true_divide



error meter: 2.644466489642394
score: 3.5826667154056953


In [170]:
#make triangle 5
CV13 = make_triangle(CV12)
for i in range(100):
    CV13 = make_triangle(CV13)
print('<<<triangle_5>>>')
CV13 = check_score(CV13)
print('--------------------')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



<<<triangle_5>>>
error meter: 2.6363950349990204
score: 3.578199946160459
--------------------


In [171]:
CV14 = move_closer_truth_train(CV13)
CV14 = check_score(CV14)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



error meter: 2.605797472865661
score: 3.479512414434079


In [172]:
#make triangle 5
CV15 = make_triangle(CV14)
for i in range(100):
    CV15 = make_triangle(CV15)
print('<<<triangle_5>>>')
CV15 = check_score(CV15)
print('--------------------')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



<<<triangle_5>>>
error meter: 2.604601324005973
score: 3.4816913382183503
--------------------


In [178]:
#guass filter
CV16 = gauss_filter(CV15)
print('<<<guass_filter>>>')
CV16 = check_score(CV16)
print('--------------------')

<<<guass_filter>>>
error meter: 2.516455229509861
score: 3.4318872733150716
--------------------


#submission

In [ ]:
#make triangle 1
sub1 = make_triangle(test_base)
for i in range(300):
    sub1 = make_triangle(sub1)
print('<<<triangle_1>>>')

#outlier
sub2 = outlier(sub1)
print('<<<outlier>>>')

#make triangle 2
sub3 = make_triangle(sub2)
for i in range(300):
    sub3 = make_triangle(sub3)
print('<<<triangle_2>>>')

#phonse mean
sub4 = mean_prediction(sub3)
print('<<<phones_mean>>>')

#make triangle 3
sub5 = make_triangle(sub4)
for i in range(300):
    sub5 = make_triangle(sub5)
print('<<<triangle_3>>>')

#remove device
sub6 = remove_device_P(sub5)
print('<<<remove_device_P>>>')

#make triangle 4
sub7 = make_triangle(sub6)
for i in range(300):
    sub7 = make_triangle(sub7)
print('<<<triangle_4>>>')

#remove device
sub8 = remove_device_S(sub7)
print('<<<remove_device_S>>>')

#make triangle 5
sub9 = make_triangle(sub8)
for i in range(300):
    sub9 = make_triangle(sub9)
print('<<<triangle_5>>>')

#position shift
sub10 = position_shift(sub9)
print('<<<position_shift>>>')

#make triangle 6
sub11 = make_triangle(sub10)
for i in range(300):
    sub11 = make_triangle(sub11)
print('<<<triangle_6>>>')

#move closer center
sub12 = move_closer_center(sub11)
print('<<<move_closer_center>>>')

#make triangle 7
sub13 = make_triangle(sub12)
for i in range(300):
    sub13 = make_triangle(sub13)
print('<<<triangle_7>>>')

#move closer truth
sub14 = move_closer_truth(sub13)
print('<<<move_closer_truth>>>')

#make triangle 8
sub15 = make_triangle(sub14)
for i in range(300):
    sub15 = make_triangle(sub15)
print('<<<triangle_8>>>')

#guass filter
sub16 = gauss_filter(sub15)
print('<<<guass_filter>>>')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars



<<<triangle_1>>>
<<<outlier>>>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



<<<triangle_2>>>



<<<phones_mean>>>
<<<triangle_3>>>
<<<remove_device_P>>>
<<<triangle_4>>>
<<<remove_device_S>>>
<<<triangle_5>>>


...

/usr/local/lib/python3.7/dist-packages/momepy/utils.py:418: UserWarning:

Approach is not set. Defaulting to 'primal'.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None




23200


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning:

invalid value encountered in true_divide



<<<move_closer_center>>>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

invalid value encountered in arccos



In [ ]:
sub12[sub.columns].to_csv("submission_71.csv", index=False)

In [ ]:
sub1

,phone,millisSinceGpsEpoch,latDeg,lngDeg,phoneName,collectionName,latDeg_from_gt,lngDeg_from_gt,latDeg_from_gt_pre,lngDeg_from_gt_pre,meter
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.416580,-122.082048,Pixel4,2020-05-15-US-MTV-1,NaN,NaN,NaN,NaN,NaN
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.416588,-122.082053,Pixel4,2020-05-15-US-MTV-1,NaN,NaN,NaN,NaN,NaN
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.416595,-122.082058,Pixel4,2020-05-15-US-MTV-1,NaN,NaN,NaN,NaN,NaN
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.416599,-122.082061,Pixel4,2020-05-15-US-MTV-1,NaN,NaN,NaN,NaN,NaN
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.416600,-122.082061,Pixel4,2020-05-15-US-MTV-1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.334544,-121.899475,SamsungS20Ultra,2021-04-29-US-SJC-3,37.334559,-121.899468,37.334559,-121.899468,1.838625
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.334547,-121.899466,SamsungS20Ultra,2021-04-29-US-SJC-3,37.334559,-121.899468,37.334559,-121.899468,1.384773
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.334549,-121.899457,SamsungS20Ultra,2021-04-29-US-SJC-3,37.334559,-121.899468,37.334559,-121.899468,1.475623
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.334551,-121.899450,SamsungS20Ultra,2021-04-29-US-SJC-3,37.334559,-121.899468,37.334559,-121.899468,1.863298
